In [1]:
import os
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary
from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

# weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

16908615680
0
0


In [2]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [4]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
#         A.Resize(640, 480)  # Resize all images to be 640x480
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [5]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(.7, .2, .1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  
    

In [6]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_file = self.imgs_files[idx]
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original, img_file
        else:
            return img, target, img_file
    
    def __len__(self):
        return len(self.imgs_files)

In [7]:
# def get_model(num_keypoints, weights_path=None):
    
#     anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
#     model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
#                                                                    pretrained_backbone=True,
#                                                                    num_keypoints=num_keypoints,
#                                                                    num_classes = 7, # Background is the first class, object is the second class
#                                                                    rpn_anchor_generator=anchor_generator)

#     if weights_path:
#         state_dict = torch.load(weights_path)
#         model.load_state_dict(state_dict)        
        
#     return model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import networkx as nx


class GNNEncoder(nn.Module):
    def __init__(self, vertices_dim=2, hidden_dim=128, num_vertices=6, num_edge_features=2):
        super(GNNEncoder, self).__init__()
        self.f_enc = nn.Linear(vertices_dim, hidden_dim)
        self.f_e1 = nn.Linear((hidden_dim * 2), hidden_dim)
        self.f_v = nn.Linear(hidden_dim, hidden_dim)
        self.f_e2 = nn.Linear((hidden_dim * 2), num_edge_features)
        self.num_vertices = num_vertices        
    
#     def get_node_features(self, vertices):
# #         print("Vertices in node features", vertices)
#         node_features = []
#         for keypoint in vertices:
#             x, y, confidence, visibility, label = keypoint
#             node_features.append([x, y, confidence, visibility, label])        
#         nodes = torch.tensor(node_features, dtype=torch.float).to(device)
# #         print(nodes)
#         return nodes

    def get_edge_features(self, vertices):
        edges = [(0,1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
#         print(edges)
#         edge_features = []
#         for edge in edges:
#             print(edge)
#             print(edge[0])
#             print(edge[1])
#             k1, k2 = vertices[edge[0]][:2], vertices[edge[1]][:2]
#             distance = torch.norm(k1 - k2)
#             angle = torch.atan2(k2[1] - k1[1], k2[0] - k1[0])
#             edge_features.append([distance.item(), angle.item()])
            
        edges = torch.tensor(edges, dtype=torch.long).to(device)
#         edge_features = torch.tensor(edge_features, dtype=torch.float).to(device)
#         return edges, edge_features
        return edges

    def forward(self, vertices):
        nodes = self.get_node_features(vertices)
        edges, edge_features = self.get_edge_features(vertices)
        h1 = self.f_enc(nodes)
        h1_source = h1[edges[:, 0]]
        h1_target = h1[edges[:, 1]]
        h_e1 = self.f_e1(torch.cat((h1_source, h1_target, edge_features), dim=1))  # Include edge feature in the input
        h_j_2 = self.f_v(h_e1)
        h2_source = h_j_2[edges[:, 0]]
        h2_target = h_j_2[edges[:, 1]]
        h_e2 = self.f_e2(torch.cat((h1_source, h1_target, edge_features), dim=1))  # Include edge feature in the input
        h_e2_prob = torch.sigmoid(h_e2)
        return vertices, h_e2_prob, edges

class GNNDecoder(nn.Module):
    def __init__(self, vertices_dim=2, hidden_dim=128, num_vertices=6, num_edge_features=2):
        super(GNNDecoder, self).__init__()
        self.f_e = nn.Linear((vertices_dim * 2), num_edge_features)  # Concatenate two vertices features
        self.f_h = nn.Linear(num_edge_features, vertices_dim)  # Transform h_ij to the same dimension as vertices
        self.f_v = nn.Linear(vertices_dim, vertices_dim)  # Update vertex feature
    
    def forward(self, vertices, h_e2_prob, edges, edge_features):
        h_source = vertices[edges[:, 0]]
        h_target = vertices[edges[:, 1]]
        h = torch.zeros_like(vertices)

        for idx, (i, j) in enumerate(edges):  # Iterate over edges
            single_edge_features = edge_features[idx].unsqueeze(0)    
            h_ij = h_e2_prob[idx] * self.f_e(torch.cat((h_source[idx].unsqueeze(0), h_target[idx].unsqueeze(0), single_edge_features), dim=1))  # Include edge weights in the input
            h_ij_transformed = self.f_h(h_ij.squeeze())  # Transform h_ij to the same dimension as vertices
            h[j] += h_ij_transformed  # Accumulate edge features to the target vertex

        h_transformed = self.f_v(h.view(-1, vertices.shape[1]))  # Transform h
        h_transformed = h_transformed.view(vertices.shape)  # Reshape back to original shape
        vertices_g = vertices + h_transformed  # Update vertex features

        return vertices_g  # Return vertices_g as the prediction and vertices_g itself as the mean for Gaussian distribution



In [ ]:
class OccludedKeyPointLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta

    def forward(self, vertices_pred, vertices_gt):
        vertices_pred = vertices_pred[:,:,:2]  # Considering only x, y coordinates, confidence_score
        vertices_gt = vertices_gt[:,:,:2]  # Considering only x, y coordinates, confidence_score
        # Compute differences
        diff = (vertices_gt - vertices_pred).abs()
        # Compute Huber loss
        huber_loss = torch.where(diff < self.delta, 0.5 * diff**2, self.delta * (diff - 0.5 * self.delta))
        return huber_loss.mean()
    
def visibility_loss (vertices_pred, vertices_gt):    
    return nn.functional.cross_entropy(vertices_pred[:,:,:2], vertices_gt[:,:,:2])  # Loss based on visibility of keypoints
def edge_loss(edges_prob, edges_gt):
    # Compute the cross-entropy loss
    loss = -torch.sum(edges_gt.to(device) * torch.log(torch.clamp(edges_prob, min=1e-7)))                      
    return loss

def temporal_consistency_loss(y_true_sequence, y_pred_sequence):
    loss = 0
    for t in range(1, len(y_true_sequence)):
        # Selecting the x, y coordinates and visibility for true and predicted sequences
        true_diff = y_true_sequence[t, :, :] - y_true_sequence[t-1, :, :]
        pred_diff = y_pred_sequence[t, :, :] - y_pred_sequence[t-1, :, :]
        loss += torch.mean(torch.abs(true_diff - pred_diff))
    return loss

In [ ]:
class KeypointPipeline(nn.Module):
    def __init__(self, num_vertices):
        super().__init__()

        self.keypoint_rcnn = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=True, num_keypoints=6, num_classes=7)
        self.num_vertices = num_vertices
        self.gnn_encoder = GNNEncoder()
        self.gnn_decoder = GNNDecoder()

    def forward(self, images, targets=None, train=False):
        if train:
            output = self.keypoint_rcnn(images, targets)
            return output
    
        else:
            all_keypoints = []
            all_edges
            with torch.no_grad():
                self.keypoint_rcnn.eval()
                output = self.keypoint_rcnn(images)
                self.keypoint_rcnn.train()

                for i in range(len(images)):  # Process each image in the batch
                    print("image in sample", i)
                    keypoints = output[i]['keypoints'].detach().cpu().numpy()
                    kp_score = output[i]['keypoints_scores'].detach().cpu().numpy()
                    labels = output[i]['labels'].detach().cpu().numpy()
                    unique_labels = list(set(labels))
                    scores = output[i]['scores'].detach().cpu().numpy()
                    print("labels", unique_labels)

                    kps = []
                    kp_scores = []
                    ulabels = []

                    for label in unique_labels:
                        indices = [j for j, x in enumerate(labels) if x == label]
                        scores_for_label = [scores[j] for j in indices]
                        max_score_index = indices[scores_for_label.index(max(scores_for_label))]
                        kp_score_label = kp_score[max_score_index].tolist()
                        kps.append(keypoints[max_score_index][kp_score_label.index(max(kp_score_label))])
                        ulabels.append(label)

                    kps = [torch.tensor(kp, dtype=torch.float32) for kp in kps]
                    if not kps:
                        default_value = torch.tensor([[320, 240, 1]], dtype=torch.float32, device=images[i].device)
                        keypoints = default_value.repeat(6, 1)
                    else:
                        keypoints = torch.stack(kps)
                        
                    print("kp before placeholder", keypoints)
                    keypoints = self.complete_missing_keypoints(keypoints, unique_labels)
                    print("kp after placeholder", keypoints)
                    vertices, self.enc_e, self.edges = self.gnn_encoder(keypoints, edge_index)
                    vertices_pred = self.gnn_decoder(vertices, self.enc_e, self.edges)
                    all_keypoints.append(vertices_pred)
                    all_pred_edges.append(self.enc_e)
                    all_edges.append(self.edges)
                    print("kp after graph", vertices_pred)
                    print("edges after graph", self.enc_e)

            print("All keypoints", all_keypoints)

            return torch.stack(all_keypoints), torch.stack(all_pred_edges), torch.stack(all_edges)
    

In [ ]:
num_epochs = 50
batch_size = 4

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = ClassDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=False)
dataset_val = ClassDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = ClassDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, pin_memory=True)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

# Initialize the GradScaler for mixed precision training
scaler = GradScaler()

top_5_models = []

model.train()

for epoch in range(num_epochs):  # for 50 epochs
    for batch_idx, batch in enumerate(data_loader_train):
        images, targets = batch  
        
        if len(images) != batch_size:
            continue
        print("The training is continued")
        
        # Move images to GPU
        images = torch.stack(images).cuda()
        # Move targets to GPU
        for target in targets:
            for key, val in target.items():
                target[key] = val.cuda()
        
        ground_truth_keypoints = [target['keypoints'] for target in targets]
        ground_truth_boxes = [target['boxes'] for target in targets]
        
        # Assuming you want all images to be of size [3, 640, 480]
#         desired_size = (640, 480)  

        # Resize all images to the desired size
#         resized_images = [F.resize(img, desired_size) for img in images]

        ground_truth_keypoints = torch.stack(ground_truth_keypoints).squeeze()[:, :, 0:2]
        print("ground_truth_keypoints", ground_truth_keypoints)
        ground_truth_boxes = torch.stack(ground_truth_boxes)[:, :, 0:2]
        
        print("ground truth keypoints shape", ground_truth_keypoints.shape)

#         Create a batched adjacency matrix with the same batch size
#         batch_adj_matrix = adj_matrix.repeat(batch_size, 1, 1)
        batch_adj_matrix = adj_matrix
        print(batch_adj_matrix.device)
        
        optimizer.zero_grad()
        
#         # Forward pass for training
#         output_train = model(images, adj_matrix=batch_adj_matrix, targets=targets, train=True)
#         print("Output keypoints shape", output_train.keys())
        
#         #Forward pass for loss
#         predicted_keypoints = model(images, adj_matrix=batch_adj_matrix, train=False)
        
        
#         print("predicted keypoints", predicted_keypoints.shape)
                
#         loss_keypoint = output_train['loss_keypoint']
        
#         # Compute loss and backpropagate
#         loss = custom_loss(predicted_keypoints, ground_truth_keypoints, 
#                            adj_matrix=batch_adj_matrix, loss_keypoint=loss_keypoint)
        
#         loss.backward()
#         optimizer.step()

        # Automatic mixed precision for forward pass
        with autocast():
            output_train = model(images, targets=targets, train=True)
#             print("Output keypoints shape", output_train.keys())
            
            predicted_keypoints = model(images, train=False)
         
#             print("predicted keypoints", predicted_keypoints)

            loss_keypoint = output_train['loss_keypoint']

            # Compute loss
            loss = custom_loss(predicted_keypoints, ground_truth_keypoints, loss_keypoint=loss_keypoint)
            
            print(loss.device)
        
        # Scale the loss and backpropagate
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scheduler.step()
        scaler.step(optimizer)
        scaler.update()

        
        # Check if the current model should be saved as a top model
        if len(top_5_models) < 5 or loss.item() < max(top_5_models, key=lambda x: x[0])[0]:
            # Save the model state and loss
            model_state = {
                'epoch': epoch,
                'complete_model': model,
#                 'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.item(),
            }
            top_5_models.append((loss.item(), model_state))

            # Sort the list based on loss (ascending order)
            top_5_models.sort(key=lambda x: x[0])

            # If there are more than 5 models, remove the one with the highest loss
            if len(top_5_models) > 5:
                top_5_models.pop()

        print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx + 1}/{len(data_loader_train)}, Loss: {loss.item()}")
        
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")
    
# After all epochs, save the top 5 models to disk
for idx, (_, model_state) in enumerate(top_5_models):
    torch.save(model_state, f'/home/jc-merlab/Pictures/Data/trained_models/best_gnn_model_b{batch_size}_e{num_epochs}_{idx+1}.pth')

In [ ]:
# Define the model
model = KeypointPipeline(num_vertices=6)
model = model.to(device)

# Define the loss
criterion = OccludedKeyPointLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 1  # Define your number of epochs
batch_size = 1

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=None, demo=False)
dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

v = 1

# # Initialize sequences for true and predicted keypoints
# y_true_sequence = []
# y_pred_sequence = []

model.train()
for epoch in range(num_epochs):
    start_time = time.time()
    for i, batch in enumerate(data_loader_train):
        img_tuple, target_dict_tuple, img_files = batch
        print(f"Processing batch {i+1} with images:", img_files)
        
        imgs = [img.to(device) for img in img_tuple]  # Create list of images

        # Process each image individually
        losses = []
        for i in range(len(imgs)):
            img = imgs[i].unsqueeze(0)  # Unsqueeze to add batch dimension

            # Prepare ground truth vertices for the image
            keypoints = target_dict_tuple[i]['keypoints'].to(device)
            print(keypoints.shape)
            # add visibility to ground truth
            visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
            print(visibility.shape)
            # add labels to ground truth
            labels = torch.arange(1, keypoints.shape[0] + 1, device=keypoints.device).view(-1, 1, 1).float()
            print(labels.shape)
            # update gt with vis and labels
            vertices_gt = torch.cat((keypoints, visibility, labels), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
            
            vertices_gt = vertices_gt.squeeze()    
            print("Ground truth keypoints from annotations", vertices_gt)
            y_true_sequence.append(vertices_gt)

            # Forward pass
            output = model(img)
            vertices_pred = output[0]
            y_pred_sequence.append(vertices_pred)
            
            edges_prob = model.enc_e
            edges = model.edges
            edge_features = model.edge_features
            edges_gt = torch.cat((edges, edge_features), dim=1) 

            # Compute loss for the image
            huber_loss = criterion(vertices_pred, vertices_gt)
            ce_loss = edge_loss(edges_prob, edges_gt)
            vis_loss = visibility_loss(vertices_pred, vertices_gt)

            loss = huber_loss + ce_loss + vis_loss
            losses.append(loss)  # Store loss for the image
            
        # Convert true and predicted sequences to tensors
        y_true_tensor = torch.stack(y_true_sequence)
        y_pred_tensor = torch.stack(y_pred_sequence)
        
        # Compute temporal consistency loss
        temporal_loss = temporal_consistency_loss(y_true_tensor, y_pred_tensor)

        # Average loss over all images in the batch
        other_losses = torch.mean(torch.stack(losses))
        
        # Combine temporal loss with other losses
        total_loss = other_losses + temporal_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Clear the sequences for the next batch
        y_true_sequence.clear()
        y_pred_sequence.clear()

    end_time = time.time()
    epoch_time = end_time - start_time
    eta = epoch_time * (num_epochs - epoch - 1)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, ETA: {eta} seconds')

model_save_path = f"/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_occ_b{batch_size}_e{num_epochs}_v{v}.pth"

torch.save(model, model_save_path)
    
# Save the state dict of the model, not the entire model
# torch.save(model.state_dict(), model_save_path)
    
torch.save(model, model_save_path)



In [ ]:
def visualize_and_save(img, vertices, filename):
    print("type of image befor conversion",type(img))    
    print("type of vertices before conversion", type(vertices))
    print(img)
    img = (img.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
#     img = (img * 255).astype(np.uint8)  # Convert back from [0, 1] range to [0, 255]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    vertices = vertices.cpu().numpy()

    print(f"Image shape before saving: {img.shape}")  # print the image shape
    print("type of vertices", type(vertices))
#     print("entered vertices", vertices)
#     print("entered image", img)

    # Convert grayscale to BGR if necessary
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    for i in range(vertices.shape[0]):
        img = cv2.circle(img, (int(vertices[i, 0]), int(vertices[i, 1])), radius=2, color=(0, 0, 255), thickness=-1)
        
    result = cv2.imwrite(filename, img)
    print(f"Image saved at {filename}: {result}")  # print if save was successful

    # If the image didn't save correctly, save the image data to a text file for examination
    if not result:
        with open(filename + ".txt", "w") as f:
            np.savetxt(f, img.flatten())

In [ ]:
def test_and_save_model(model, data_loader_test):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_trifocal_loss = 0.0
    total_ce_loss = 0.0
    total_vis_loss = 0.0
    num_batches = 0

    y_true_sequence = []
    y_pred_sequence = []

    # We don't need to track gradients during evaluation
    with torch.no_grad():
        for idx, batch in enumerate(data_loader_test):
            img_tuple, target_dict_tuple, img_files = batch

            total_batch_loss = 0.0
            total_batch_trifocal_loss = 0.0
            total_batch_ce_loss = 0.0
            total_batch_vis_loss = 0.0

            # Process each image individually
            for i in range(len(img_tuple)):
                img = img_tuple[i].to(device)
                target = target_dict_tuple[i]

                # Prepare ground truth vertices for the image
                keypoints = target['keypoints'].to(device)
                visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
                vertices_gt = torch.cat((keypoints, visibility), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
                vertices_gt = vertices_gt.squeeze()
                y_true_sequence.append(vertices_gt)

                # Forward pass
                output = model(img.unsqueeze(0))
                vertices_pred = output[0]
                y_pred_sequence.append(vertices_pred)

                edges_prob = model.enc_e
                edges = model.edges
                edge_features = model.edge_features
                edges_gt = torch.cat((edges, edge_features), dim=1) 

                trifocal_loss = criterion(vertices_pred, vertices_gt)
                ce_loss = edge_loss(edges_prob, edges_gt)
                vis_loss = visibility_loss(vertices_pred, vertices_gt)
                loss = trifocal_loss + ce_loss + vis_loss

                total_batch_loss += loss.item()
                total_batch_trifocal_loss += trifocal_loss.item()
                total_batch_ce_loss += ce_loss.item()
                total_batch_vis_loss += vis_loss.item()

                # Visualize and save the prediction
                filename = f'/home/jc-merlab/Pictures/Data/occ_vis_data/image_{idx}_{i}.jpg'
                visualize_and_save(img, vertices_pred, filename)
                print(f"Image saved at {filename}")  # Print statement to confirm image save

            # Convert true and predicted sequences to tensors
            y_true_tensor = torch.stack(y_true_sequence)
            y_pred_tensor = torch.stack(y_pred_sequence)

            # Compute temporal consistency loss
            temporal_loss = temporal_consistency_loss(y_true_tensor, y_pred_tensor)

            total_loss += (total_batch_loss + temporal_loss) / len(img_tuple)
            total_trifocal_loss += total_batch_trifocal_loss / len(img_tuple)
            total_ce_loss += total_batch_ce_loss / len(img_tuple)
            num_batches += 1

            # Clear the sequences for the next batch
            y_true_sequence.clear()
            y_pred_sequence.clear()
    
    # Average the loss over all batches
    avg_loss = total_loss / num_batches
    avg_trifocal_loss = total_trifocal_loss / num_batches
    avg_ce_loss = total_ce_loss / num_batches
    
    print(f'Avg. Test Loss: {avg_loss}, Avg. Trifocal Loss: {avg_trifocal_loss}, Avg. Cross Entropy Loss: {avg_ce_loss}')
    return avg_loss, avg_trifocal_loss, avg_ce_loss

In [ ]:
# avg_loss, avg_trifocal_loss, avg_ce_loss, all_preds = test_and_save_model(model, data_loader_test)

avg_loss, avg_trifocal_loss, avg_ce_loss = test_and_save_model(model, data_loader_test)

In [ ]:
import cv2
import os

# Directory containing images
dir_path = '/home/jc-merlab/Pictures/Data/occ_vis_data/'
images = []

# Ensure the images are sorted by name
for f in sorted(os.listdir(dir_path)):
    if f.endswith('.jpg') or f.endswith('.png'):  # Check for image file extension
        images.append(f)

# Determine the width and height from the first image
image_path = os.path.join(dir_path, images[0])
frame = cv2.imread(image_path)
cv2.imshow('video',frame)
height, width, channels = frame.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Be sure to use the correct codec
video_filename = 'output.mp4'
video = cv2.VideoWriter(video_filename, fourcc, 3.0, (width, height))

for image in images:
    image_path = os.path.join(dir_path, image)
    frame = cv2.imread(image_path)
    video.write(frame)  # Write out frame to video

# Release everything when job is finished
video.release()
cv2.destroyAllWindows()

print("The output video is", video_filename)

In [ ]:
model_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_occ_b16_e25_v6.pth'

model = torch.load(model_path).to(device)


image = Image.open("/home/jc-merlab/Pictures/Data/occluded_results_mi20_ma80_n2/occluded_000041.rgb.jpg")
print(type(image))

img = F.to_tensor(image).to(device)
img.unsqueeze_(0)
# print(image.shape)
# image = list(image)
# print(type(images))
# images = list(image.to(device) for image in images)

with torch.no_grad():
    model.to(device)
    model.eval()
    output = model(img)
    
keypoints = output[0]

print(keypoints)
plt.imshow(image)

# Assuming each keypoint is a tensor representing (x, y)
for i, keypoint in enumerate(keypoints):
    print(f'Key point {i}: {keypoint}')
    keypoint = keypoint.cpu().numpy()
    plt.plot(keypoint[0], keypoint[1], 'ro')
plt.show()

# Plotting the image

# plt.imshow(image)

# for keypoint in output[0]:
#     plt.plot(keypoint[0], keypoint[1], 'ro')

# plt.show()